In [ ]:
!pip install yfinance py_vollib

# Data Retrieval: 
Fetch historical data for SPY.

In [22]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import requests

In [65]:
def fetch_and_process_data(_asset):
    # Fetch data for the specified asset
    hist = yf.download(_asset, start='2022-01-01') 

    # Indicator calculations as defined earlier
    def bollinger_bands(data, window=20, num_std=2):
        rolling_mean = data['Close'].rolling(window=window).mean()
        rolling_std = data['Close'].rolling(window=window).std()
        data['Bollinger_High'] = rolling_mean + (rolling_std * num_std)
        data['Bollinger_Low'] = rolling_mean - (rolling_std * num_std)
        return data

    def macd(data, short_window=12, long_window=26, signal_window=9):
        short_ema = data['Close'].ewm(span=short_window, adjust=False).mean()
        long_ema = data['Close'].ewm(span=long_window, adjust=False).mean()
        data['MACD'] = short_ema - long_ema
        data['Signal'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
        return data

    def rsi(data, periods=14, ema=True):
        close_delta = data['Close'].diff()
        up = close_delta.clip(lower=0)
        down = -1 * close_delta.clip(upper=0)
        
        if ema:
            ma_up = up.ewm(com=periods - 1, adjust=True, min_periods=periods).mean()
            ma_down = down.ewm(com=periods - 1, adjust=True, min_periods=periods).mean()
        else:
            ma_up = up.rolling(window=periods, adjust=False).mean()
            ma_down = down.rolling(window=periods, adjust=False).mean()
        
        rsi = ma_up / ma_down
        data['RSI'] = 100 - (100 / (1 + rsi))
        return data
        
    def obv(data):
        """Calculate On-Balance Volume."""
        obv = (np.sign(data['Close'].diff()) * data['Volume']).fillna(0).cumsum()
        data['OBV'] = obv
        return data

    def atr(data, window=14):
        """Calculate Average True Range (ATR)."""
        high_low = data['High'] - data['Low']
        high_close = np.abs(data['High'] - data['Close'].shift())
        low_close = np.abs(data['Low'] - data['Close'].shift())
        ranges = pd.concat([high_low, high_close, low_close], axis=1)
        true_range = np.max(ranges, axis=1)
        data['ATR'] = true_range.rolling(window=window).mean()
        return data

    def woodie_pivots(data):
        # Calculate Woodie's pivot points
        data['Pivot'] = (data['High'] + data['Low'] + 2 * data['Close']) / 4
        data['R1'] = 2 * data['Pivot'] - data['Low']
        data['S1'] = 2 * data['Pivot'] - data['High']
        data['R2'] = data['Pivot'] + (data['High'] - data['Low'])
        data['S2'] = data['Pivot'] - (data['High'] - data['Low'])
        data['R3'] = data['High'] + 2 * (data['Pivot'] - data['Low'])
        data['S3'] = data['Low'] - 2 * (data['High'] - data['Pivot'])
        data['R4'] = data['Pivot'] + 3 * (data['High'] - data['Low'])
        data['S4'] = data['Pivot'] - 3 * (data['High'] - data['Low'])
        return data

    # Apply each indicator function to the data
    hist = bollinger_bands(hist)
    hist = macd(hist)
    hist = rsi(hist)
    hist = woodie_pivots(hist)
    hist = obv(hist)
    hist = atr(hist)
    # Repeat for other indicators as necessary...

    # Note: No explicit parallel processing applied here due to sequential dependency of calculations on data.

    # Ensure all NaN values created by indicators are handled appropriately
    hist.dropna(inplace=True)

    return hist

# Example usage
spy_data = fetch_and_process_data("SPY")
print(spy_data.tail())  # Display the last few rows to verify the outcome
# spy_data

[*********************100%%**********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2024-02-13  494.529999  497.089996  490.720001  494.079987  494.079987   
2024-02-14  496.790009  499.070007  494.399994  498.570007  498.570007   
2024-02-15  499.290009  502.200012  498.799988  502.010010  502.010010   
2024-02-16  501.700012  502.869995  498.750000  499.510010  499.510010   
2024-02-20  497.720001  498.410004  494.459991  496.760010  496.760010   

               Volume  Bollinger_High  Bollinger_Low      MACD    Signal  ...  \
Date                                                                      ...   
2024-02-13  113099200      504.900087     473.914919  6.587714  6.426473  ...   
2024-02-14   68387800      504.459707     476.983298  6.468158  6.434810  ...   
2024-02-15   61683000      504.884360     479.110647  6.575193  6.462887  ...   
2024-02

# Check Data For any errors:

In [35]:
from pandas.tseries.holiday import USFederalHolidayCalendar

def check_data_errors(data):
    errors = []

    # Check for missing values
    if data.isnull().values.any():
        errors.append("Issue: Data contains missing values.")
    
    # Check for duplicate dates
    if data.index.duplicated().any():
        errors.append("Issue: Data contains duplicate dates.")
    
    # Outliers in price data
    z_scores = np.abs((data['Close'] - data['Close'].mean()) / data['Close'].std())
    if z_scores[z_scores > 3].any():
        errors.append("Issue: Data contains potential outliers in 'Close' prices.")
    
    # Volume checks
    if (data['Volume'] == 0).any():
        errors.append("Issue: Data contains days with zero volume.")
    if ((data['Volume'].diff() / data['Volume']).abs() > 5).any():
        errors.append("Issue: Data contains unexpected spikes in volume.")
    
    # Continuity of dates, excluding weekends and public holidays
    cal = USFederalHolidayCalendar()
    holidays = cal.holidays(start=data.index.min(), end=data.index.max())
    business_days = pd.date_range(start=data.index.min(), end=data.index.max(), freq='B')
    business_days = business_days[~business_days.isin(holidays)]  # Exclude holidays
    
    missing_dates = business_days.difference(data.index).tolist()
    if missing_dates:
        formatted_dates = ', '.join([d.strftime('%Y-%m-%d') for d in missing_dates])
        errors.append(f"Issue: Data might be missing trading days: {formatted_dates}")

    return errors

# Example usage
spy_data = fetch_and_process_data("SPY")  # Assuming this function returns data with DateTimeIndex
errors = check_data_errors(spy_data)
if errors:
    for error in errors:
        print(error)
else:
    print("No issues detected in the data.")


[*********************100%%**********************]  1 of 1 completed
Issue: Data might be missing trading days: 2022-04-15, 2023-04-07


The error message indicating missing trading days for specific dates such as April 19, 2019, April 10, 2020, April 2, 2021, April 15, 2022, and April 7, 2023, highlights dates that are actually Good Friday. In the United States, the stock market (NYSE, NASDAQ) is closed on Good Friday, which is not a federal holiday and therefore not included in the USFederalHolidayCalendar. This explains why these dates were flagged as missing trading days by the previous function.

To address this and accurately reflect the trading calendar, we need to manually account for Good Friday and potentially other market-specific closures not covered by the federal holiday calendar. Here's an updated version of the function that checks for missing trading days, now including an adjustment for Good Friday and a more general approach to handling non-trading days:

Frankly, this is good enough for 

# Credit Spread Pricing
Here we want to use R1-4 and S1-S4 to find the nearest 50 cent spread.
We'll then try to determine the probablity of each expiring.

In [73]:
import pandas as pd
import numpy as np

def round_to_nearest_50_cents(value):
    """Round the value to the nearest 50 cents."""
    return np.round(value * 2, 0) / 2

def generate_credit_spreads(data):
    # Initialize a list to hold the spreads for each day
    spreads = []
    
    # Iterate through each row in the DataFrame
    for index, row in data.iterrows():
        # Initialize a dictionary for the current day's spreads
        day_spreads = {
            'Date': index,
            'R1-R2 Put Spread': None,
            'R1-R3 Put Spread': None,
            'R1-R4 Put Spread': None,
            'S1-S2 Call Spread': None,
            'S1-S3 Call Spread': None,
            'S1-S4 Call Spread': None,
        }
        
        # Calculate the put credit spreads for each pair of resistances
        for i in range(2, 5):
            sell_strike = round_to_nearest_50_cents(row['R1'])
            buy_strike = round_to_nearest_50_cents(row[f'R{i}'])
            day_spreads[f'R1-R{i} Put Spread'] = (sell_strike, buy_strike)
        
        # Calculate the call credit spreads for each pair of supports
        for i in range(2, 5):
            sell_strike = round_to_nearest_50_cents(row['S1'])
            buy_strike = round_to_nearest_50_cents(row[f'S{i}'])
            day_spreads[f'S1-S{i} Call Spread'] = (sell_strike, buy_strike)
        
        # Add the current day's spreads to the list
        spreads.append(day_spreads)
    
    # Convert the list of spreads into a DataFrame for easier viewing
    spreads_df = pd.DataFrame(spreads)
    spreads_df.set_index('Date', inplace=True)
    return spreads_df

# Assuming spy_data is already defined and contains the necessary columns
# Generate the credit spreads
credit_spreads = generate_credit_spreads(spy_data)
print(credit_spreads)
credit_spreads

           R1-R2 Put Spread R1-R3 Put Spread R1-R4 Put Spread  \
Date                                                            
2022-01-31   (455.0, 458.0)   (455.0, 465.5)   (455.0, 479.0)   
2022-02-01   (456.5, 458.5)   (456.5, 463.0)   (456.5, 471.5)   
2022-02-02   (460.0, 461.5)   (460.0, 465.0)   (460.0, 471.5)   
2022-02-03   (450.0, 455.0)   (450.0, 457.5)   (450.0, 470.0)   
2022-02-04   (453.0, 457.5)   (453.0, 462.0)   (453.0, 475.5)   
...                     ...              ...              ...   
2024-02-13   (497.5, 500.5)   (497.5, 503.5)   (497.5, 513.0)   
2024-02-14   (501.0, 502.5)   (501.0, 505.5)   (501.0, 511.5)   
2024-02-15   (503.5, 504.5)   (503.5, 507.0)   (503.5, 511.5)   
2024-02-16   (501.5, 504.5)   (501.5, 505.5)   (501.5, 512.5)   
2024-02-20   (498.5, 500.5)   (498.5, 502.5)   (498.5, 508.5)   

           S1-S2 Call Spread S1-S3 Call Spread S1-S4 Call Spread  
Date                                                              
2022-01-31    (444.5

,R1-R2 Put Spread,R1-R3 Put Spread,R1-R4 Put Spread,S1-S2 Call Spread,S1-S3 Call Spread,S1-S4 Call Spread
Date,,,,,,
2022-01-31,"(455.0, 458.0)","(455.0, 465.5)","(455.0, 479.0)","(444.5, 437.0)","(444.5, 434.0)","(444.5, 416.0)"
2022-02-01,"(456.5, 458.5)","(456.5, 463.0)","(456.5, 471.5)","(449.5, 445.0)","(449.5, 443.0)","(449.5, 431.5)"
2022-02-02,"(460.0, 461.5)","(460.0, 465.0)","(460.0, 471.5)","(455.0, 451.5)","(455.0, 449.5)","(455.0, 441.5)"
2022-02-03,"(450.0, 455.0)","(450.0, 457.5)","(450.0, 470.0)","(443.0, 440.5)","(443.0, 435.5)","(443.0, 426.0)"
2022-02-04,"(453.0, 457.5)","(453.0, 462.0)","(453.0, 475.5)","(444.0, 439.5)","(444.0, 435.5)","(444.0, 421.5)"
...,...,...,...,...,...,...
2024-02-13,"(497.5, 500.5)","(497.5, 503.5)","(497.5, 513.0)","(491.0, 487.5)","(491.0, 484.5)","(491.0, 475.0)"
2024-02-14,"(501.0, 502.5)","(501.0, 505.5)","(501.0, 511.5)","(496.0, 493.0)","(496.0, 491.5)","(496.0, 483.5)"
2024-02-15,"(503.5, 504.5)","(503.5, 507.0)","(503.5, 511.5)","(500.5, 498.0)","(500.5, 497.0)","(500.5, 491.0)"


# Binomial Tree
Implementation of a simple slow and fast binomial pricing model in python. We will treat binomial tree as a network with nodes (i,j) with i representing the time steps and j representing the number of ordered price outcome (lowest - or bottom of tree - to highest).

- american_tree_slow
- american_tree_fast

### Binomial Tree Representation

Stock tree can be represented using nodes (i,j) and intial stock price $S_0$

$S_{i,j} = S_0u^{j}d^{i-j}$

$C_{i,j}$ represents contract price at each node (i,j). Where $C_{N,j}$ represents final payoff function that we can define.

In [75]:
from functools import wraps
from time import time

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func:%r args:[%r, %r] took: %2.4f sec' % \
          (f.__name__, args, kw, te-ts))
        return result
    return wrap

### American Option Characteristics
For an American Put Option:

if $T = t_N$ then at the terminal nodes, $C^{j}_N = (K-S^{j}_N)^{+}$

for all other parts of the tree at nodes $(i,j)$

- Max of exercise value or continuation/hold value

- $C^{j}_i = \max \Big((K-S^{j}_i)^{+}, \exp^{-r\Delta t} \big\{ q^{j}_i C^{j+1}_{i+1} + (1 - q^{j}_i)C^{j-1}_{i-1}\big\}\Big)$

In [76]:
# Initialise parameters
S0 = 100      # initial stock price
K = 100       # strike price
T = 1         # time to maturity in years
r = 0.06      # annual risk-free rate
N = 3         # number of time steps
u = 1.1       # up-factor in binomial models
d = 1/u       # ensure recombining tree
opttype = 'P' # Option Type 'C' or 'P'

### American Tree Slow

Here we will use for loops to iterate through nodes j at each time step i.

In [77]:
 @timing
def american_slow_tree(K,T,S0,r,N,u,d,opttype='P'):
    #precompute values
    dt = T/N
    q = (np.exp(r*dt) - d)/(u-d)
    disc = np.exp(-r*dt)

    # initialise stock prices at maturity
    S = np.zeros(N+1)
    for j in range(0, N+1):
        S[j] = S0 * u**j * d**(N-j)

    # option payoff
    C = np.zeros(N+1)
    for j in range(0, N+1):
        if opttype == 'P':
            C[j] = max(0, K - S[j])
        else:
            C[j] = max(0, S[j] - K)

    # backward recursion through the tree
    for i in np.arange(N-1,-1,-1):
        for j in range(0,i+1):
            S = S0 * u**j * d**(i-j)
            C[j] = disc * ( q*C[j+1] + (1-q)*C[j] )
            if opttype == 'P':
                C[j] = max(C[j], K - S)
            else:
                C[j] = max(C[j], S - K)

    return C[0]

american_slow_tree(K,T,S0,r,N,u,d,opttype='P')

func:'american_slow_tree' args:[(100, 1, 100, 0.06, 3, 1.1, 0.9090909090909091), {'opttype': 'P'}] took: 0.0010 sec


4.654588754602527

### American Tree Fast

Now we will vectorise out code using numpy arrays instead of for loops through j nodes.

In [78]:
@timing
def american_fast_tree(K,T,S0,r,N,u,d,opttype='P'):
    #precompute values
    dt = T/N
    q = (np.exp(r*dt) - d)/(u-d)
    disc = np.exp(-r*dt)

    # initialise stock prices at maturity
    S = S0 * d**(np.arange(N,-1,-1)) * u**(np.arange(0,N+1,1))

    # option payoff
    if opttype == 'P':
        C = np.maximum(0, K - S)
    else:
        C = np.maximum(0, S - K)

    # backward recursion through the tree
    for i in np.arange(N-1,-1,-1):
        S = S0 * d**(np.arange(i,-1,-1)) * u**(np.arange(0,i+1,1))
        C[:i+1] = disc * ( q*C[1:i+2] + (1-q)*C[0:i+1] )
        C = C[:-1]
        if opttype == 'P':
            C = np.maximum(C, K - S)
        else:
            C = np.maximum(C, S - K)

    return C[0]

american_fast_tree(K,T,S0,r,N,u,d,opttype='P')

func:'american_fast_tree' args:[(100, 1, 100, 0.06, 3, 1.1, 0.9090909090909091), {'opttype': 'P'}] took: 0.0000 sec


4.654588754602527

### Binomial Tree Slow vs Fast

Now we will compare runtimes for slow vs fast. Ignore option price changes as this is impacted with changing the time steps and keeping the u and d factors the same.

In [80]:
for N in [3,50, 100, 1000, 5000]:
    american_fast_tree(K,T,S0,r,N,u,d,opttype='P')
    american_slow_tree(K,T,S0,r,N,u,d,opttype='P')

func:'american_fast_tree' args:[(100, 1, 100, 0.06, 3, 1.1, 0.9090909090909091), {'opttype': 'P'}] took: 0.0000 sec
func:'american_slow_tree' args:[(100, 1, 100, 0.06, 3, 1.1, 0.9090909090909091), {'opttype': 'P'}] took: 0.0000 sec
func:'american_fast_tree' args:[(100, 1, 100, 0.06, 50, 1.1, 0.9090909090909091), {'opttype': 'P'}] took: 0.0010 sec
func:'american_slow_tree' args:[(100, 1, 100, 0.06, 50, 1.1, 0.9090909090909091), {'opttype': 'P'}] took: 0.0030 sec
func:'american_fast_tree' args:[(100, 1, 100, 0.06, 100, 1.1, 0.9090909090909091), {'opttype': 'P'}] took: 0.0025 sec
func:'american_slow_tree' args:[(100, 1, 100, 0.06, 100, 1.1, 0.9090909090909091), {'opttype': 'P'}] took: 0.0112 sec
func:'american_fast_tree' args:[(100, 1, 100, 0.06, 1000, 1.1, 0.9090909090909091), {'opttype': 'P'}] took: 0.0414 sec
func:'american_slow_tree' args:[(100, 1, 100, 0.06, 1000, 1.1, 0.9090909090909091), {'opttype': 'P'}] took: 1.2736 sec
func:'american_fast_tree' args:[(100, 1, 100, 0.06, 5000, 1.

In [87]:
import pandas as pd
import numpy as np
from scipy.stats import norm
print(spy_data)
print(credit_spreads)
def calculate_option_probabilities(spy_data, credit_spreads, volatility, risk_free_rate, time_to_expiration):
    """
    Calculate the probabilities of credit spreads expiring worthless using the Black-Scholes model.
    
    Parameters:
    - spy_data: DataFrame containing 'Close' prices of the stock.
    - credit_spreads: DataFrame containing the strike prices for the spreads.
    - volatility: Annualized volatility of the stock.
    - risk_free_rate: Annual risk-free interest rate.
    - time_to_expiration: Time to expiration of the options in years.
    
    Returns:
    - DataFrame with probabilities of each spread expiring worthless.
    """
    def calculate_cdf(strike, current_price, volatility, time_to_expiration, risk_free_rate):
        """Calculate the cumulative distribution function for Black-Scholes."""
        d1 = (np.log(current_price / strike) + (risk_free_rate + 0.5 * volatility ** 2) * time_to_expiration) / (volatility * np.sqrt(time_to_expiration))
        return norm.cdf(d1)
    
    probabilities = []
    
    for index, row in spy_data.iterrows():
        current_price = row['Close']
        
        # Adjusted to directly use 'R' and 'S' values from `credit_spreads`
        for i in range(1, 5):
            if f'R{i}' in credit_spreads.columns and f'S{i}' in credit_spreads.columns:
                # For put spreads, using R values
                sell_strike_put = row[f'R{i}']
                buy_strike_put = row[f'R{i}'] - 1  # Example adjustment, customize as needed
                prob_put = calculate_cdf(sell_strike_put, current_price, volatility, time_to_expiration, risk_free_rate)
                
                # For call spreads, using S values
                sell_strike_call = row[f'S{i}']
                buy_strike_call = row[f'S{i}'] + 1  # Example adjustment, customize as needed
                prob_call = 1 - calculate_cdf(buy_strike_call, current_price, volatility, time_to_expiration, risk_free_rate)
                
                probabilities.append({
                    'Date': index,
                    f'R{i} Put Spread Probability': prob_put,
                    f'S{i} Call Spread Probability': prob_call
                })
    
    probabilities_df = pd.DataFrame(probabilities).set_index('Date')
    return probabilities_df

# Assuming you have calculated or have the values for the following variables:
volatility = 0.2  # Example volatility
risk_free_rate = 0.01  # Example risk-free rate
time_to_expiration = 1/52  # 1 week to expiration

# Calculate probabilities
probabilities_df = calculate_option_probabilities(spy_data, spy_data, volatility, risk_free_rate, time_to_expiration)
print(probabilities_df)


                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2022-01-31  441.239990  450.279999  439.809998  449.910004  436.099396   
2022-02-01  450.679993  453.630005  446.940002  452.950012  439.046112   
2022-02-02  455.500000  458.119995  453.049988  457.350006  443.311005   
2022-02-03  450.950012  452.970001  445.709991  446.600006  432.890991   
2022-02-04  446.350006  452.779999  443.829987  448.700012  434.926544   
...                ...         ...         ...         ...         ...   
2024-02-13  494.529999  497.089996  490.720001  494.079987  494.079987   
2024-02-14  496.790009  499.070007  494.399994  498.570007  498.570007   
2024-02-15  499.290009  502.200012  498.799988  502.010010  502.010010   
2024-02-16  501.700012  502.869995  498.750000  499.510010  499.510010   
2024-02-20  497.720001  498.410004  494.459991  496.760010  496.760010   

               Volume  Bollinger_High